In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import spacy

from sklearn.metrics import *
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load spacy nlp

In [ ]:
nlp = nlp = spacy.load("en_core_web_sm")

## Load Transformed data

In [ ]:
df = pd.read_csv('/kaggle/input/impact-of-covid19-pandemic-on-the-global-economy/transformed_data.csv')
df = df.drop(columns=['CODE'])
df.head(), df.shape

## Basic plot of all data

In [ ]:
plt.figure(figsize=(20, 15))
df.plot()

## Preprocessing Code, Country and Date to Word Vectors, and spliting data to test and train

In [ ]:
df.fillna(0, inplace=True)
df['DATE'] = df['DATE'].apply(lambda x: pd.to_datetime(x).value)
for i in range(50418):
    df.COUNTRY[i] = [token.vector for token in nlp(df.COUNTRY[i])]

In [ ]:
df['DATE'] *= 1e-15
df['COUNTRY'] = df['COUNTRY'].apply(np.sum)
X, y = df.iloc[:, :7], df['GDPCAP']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train

In [ ]:
y_train

## Model gdpcap (dosmestic gross)

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

samples, features = X_train.shape
input_size = features
output_size = 1

samples, features

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        
        self.l1 = nn.Linear(input_size, 500)
        self.l2 = nn.Linear(500, 200)
        self.l3 = nn.Linear(200, output_size)
        
    def forward(self, x):
        
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l3(x)
        return x

In [ ]:
model = Net(input_size, output_size)

lr = 0.00001
epochs = 100
loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
for epoch in range(epochs):
    
    pred = model(X_train)
    l = loss(y_train, pred)
    l.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % 10 == 0:
        print(f'epoch {epoch + 1}, loss = {l}')

In [ ]:
model(X_test) - y_test